# RG Analysis: $\alpha$ versus $\beta$ Phase Diagram

We will do the RG plot for $\alpha$ versus $\beta$ here, throughout RG steps. We will do this only for one step, since the plot is cleaner that way.
Since we know that $\beta$ values remain roughly constant from our previous simulations, we need not learn it here. The values are more accurate - 3 parameter optimization is often more difficult. We can clearly see the phase transition happen close to $\beta_0^2 = 8\pi$.

In [ ]:
using ColorSchemes
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, JuMP, Ipopt, Plots, Optim, GalacticOptim
using Logging
using StatsBase: autocor
# ========== Model Definition ==========

struct SineGordonModel2D
    lattice_size::Int  # Lattice size (NxN)
    a::Float64         # Coupling strength for the sine term
    b::Float64         # Mass parameter squared (m^2)
    Z::Float64         # Wavefunction renormalization term
end

# ========== Hamiltonian / Energy ==========

function calculate_energy_2d(phi::Matrix{T}, model::SineGordonModel2D) where T<:Real
    nx, nt = size(phi)
    Z = model.Z
    kinetic_energy = zero(T)
    gradient_energy = zero(T)
    sine_energy = zero(T)
    for x in 1:nx
        for t in 1:nt
            t_prev = mod1(t - 1, nt)
            kinetic_energy += T(0.5) * Z * ((phi[x, t] - phi[x, t_prev]))^2
            x_prev = mod1(x - 1, nx)
            gradient_energy += T(0.5) * Z * ((phi[x, t] - phi[x_prev, t]))^2
            sine_energy += (model.a) * (1 - cos(model.b * phi[x, t]))
        end
    end
    return kinetic_energy + gradient_energy + sine_energy
end

# ========== Log Density Wrapper ==========

struct SineGordonModelDensity2D
    dim::Int
    model::SineGordonModel2D
end

function LogDensityProblems.logdensity(p::SineGordonModelDensity2D, phi_vector::Vector{T}) where T<:Real
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::SineGordonModelDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{SineGordonModelDensity2D}) = LogDensityProblems.LogDensityOrder{1}()

# ========== HMC Sampling ==========

using MCMCChains, StatsBase

function generate_SG_data_hmc_2d(
    model::SineGordonModel2D,
    n_samples::Int,
    n_adapts::Int = 5000;
    burn_in::Int = 1000,
    thin::Int = 10,
    n_checksites::Int = 5
)
    lattice_size = model.lattice_size
    sg_target = SineGordonModelDensity2D(lattice_size^2, model)
    initial_phi = randn(lattice_size, lattice_size)

    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, sg_target, ForwardDiff)
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    # total draws = requested * thinning + burn-in
    total_draws = n_samples * thin + burn_in
    raw_samples, _ = with_logger(NullLogger()) do
        sample(
            hamiltonian,
            kernel,
            vec(initial_phi),
            total_draws,
            adaptor,
            n_adapts;
            progress = false
        )
    end

    # Discard burn-in, then thin
    filtered = raw_samples[(burn_in+1):thin:end]

    samples = [reshape(sample, lattice_size, lattice_size) for sample in filtered]

    # ----------------------------
    # Diagnostics (autocorr + ESS)
    # ----------------------------
    chain_mat = reduce(hcat, [vec(s) for s in samples])' # each row = one sample
    ns, d = size(chain_mat)
    println("Collected $ns effective samples after thinning.")

    # choose random sites to check
    site_idxs = rand(1:d, min(n_checksites,d))
    for (i, idx) in enumerate(site_idxs)
        site_series = chain_mat[:, idx]
        ess_val = ess_custom(site_series; maxlag=50)
        println("Site $idx: ESS = $ess_val")
    end

    return samples
end



# ========== Score Matching Objective ==========

function score_matching_objective_2d(a::T, b::T, Z::Float64, data::Vector{Matrix{Float64}}) where T<:Real
    nx, nt = size(data[1])
    n_samples = length(data)
    obj = zero(T)
    for sample in data
        for x in 1:nx
            for t in 1:nt
                phi_xt = sample[x, t]
                t_prev = mod1(t - 1, nt)
                x_prev = mod1(x - 1, nx)
                t_next = mod1(t + 1, nt)
                x_next = mod1(x + 1, nx)
                sum_neighbor_phi = sample[x, t_prev] + sample[x, t_next] + sample[x_prev, t] + sample[x_next, t]
                score_xt = -(Z * (4 * phi_xt - sum_neighbor_phi) + (a * b) * sin(b * phi_xt))
                s_prime = - (4 * Z + a * b^2 * cos(b * phi_xt))
                obj += T(0.5) * score_xt^2 + s_prime
            end
        end
    end
    return obj / n_samples
end

# ========== Parameter Estimation (α and Z only) ==========

function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}};
                                               a_bounds=(0.001,10.0),
                                               b_bounds=(5.0,6.0),
                                               Z_bounds=(0.001,10.0))
    # bounds for [a, b, Z]
    lower = [a_bounds[1], b_bounds[1], Z_bounds[1]]
    upper = [a_bounds[2], b_bounds[2], Z_bounds[2]]

    # objective as a function of x = [a, b, Z]
    f(x) = score_matching_objective_2d(x[1], x[2], x[3], data)

    # start in the middle of the box
    initial_x = [(lower[i] + upper[i]) / 2 for i in 1:3]

    # use Particle Swarm (or any 3‐D optimizer you like)
    ps = ParticleSwarm()
    result = optimize(f, lower, upper, initial_x, ps)

    a_opt, b_opt, Z_opt = result.minimizer
    println("Estimated: a=$(a_opt), β=$(b_opt), Z=$(Z_opt)")
    return a_opt, b_opt, Z_opt
end

#= function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})
    # objective as a function of x = [a, b, Z]
    f(x) = score_matching_objective_2d(x[1], x[2], x[3], data)

    # initial guess (you can change this if needed)
    initial_x = [1.0, 1.0, 1.0]

    # use an unconstrained optimizer like Nelder-Mead
    result = optimize(f, initial_x, NelderMead())

    a_opt, b_opt, Z_opt = result.minimizer
    println("Estimated: a=$(a_opt), β=$(b_opt), Z=$(Z_opt)")
    return a_opt, b_opt, Z_opt
end =#

# ========== Block Spin ==========

function block_spin_transform(phi::Matrix{Float64})
    nx, ny = size(phi)
    if nx % 2 != 0 || ny % 2 != 0
        error("Lattice size must be even for block spin transformation.")
    end
    nx_new = div(nx, 2)
    ny_new = div(ny, 2)
    phi_new = zeros(Float64, nx_new, ny_new)
    for i in 1:nx_new
        for j in 1:ny_new
            block = phi[2i-1:2i, 2j-1:2j]
            phi_new[i, j] = median(vec(block))
        end
    end
    return phi_new / sqrt(2)
end

function block_spin_samples(samples::Vector{Matrix{Float64}})
    return [block_spin_transform(sample) for sample in samples]
end

# ========== RG Flow with Constant β ==========

function run_a_vs_b_trajectory(
    a_0::Float64, b_0::Float64, Z_0::Float64,
    L::Int, n_samples::Int,
    n_steps::Int, n_rep::Int
)
    as_reps = zeros(n_rep, n_steps)
    bs_reps = zeros(n_rep, n_steps)

    for rep in 1:n_rep
        # 1) start
        model   = SineGordonModel2D(L, a_0, b_0, Z_0)
        samples = generate_SG_data_hmc_2d(model, n_samples)

        a_traj = [a_0]
        b_traj = [b_0]

        # 2) do the RG steps
        for step in 1:(n_steps - 1)
            samples = block_spin_samples(samples)
            a_est, b_est, _ = estimate_parameters_score_matching_2d(samples)
            push!(a_traj, abs(a_est))
            push!(b_traj, abs(b_est))
        end

        as_reps[rep, :] = a_traj
        bs_reps[rep, :] = b_traj
    end

    # return means & stds as before
    as_mean = [mean(as_reps[:, i]) for i in 1:n_steps]
    as_std  = [std(as_reps[:,  i]) for i in 1:n_steps]
    bs_mean = [mean(bs_reps[:, i]) for i in 1:n_steps]
    bs_std  = [std(bs_reps[:,  i]) for i in 1:n_steps]

    return as_mean, as_std, bs_mean, bs_std
end

# ========== Plotting ==========

function scan_and_plot_a_vs_b(
    a_fixed::Float64,
    b_range::Vector{Float64},
    Z_fixed::Float64,
    lattice_size::Int,
    n_samples::Int,
    n_steps::Int,
    n_rep::Int
)
    plot(
      xlabel = "β₀",
      ylabel = "α",
      title  = "RG α‐flows at fixed β₀ (mean ± std over $n_rep reps)",
      legend = :outertopright,
    )

    for (ci, b0) in enumerate(b_range)
        as_mean, as_std, _, _ = run_a_vs_b_trajectory(
            a_fixed, b0, Z_fixed,
            lattice_size, n_samples,
            n_steps, n_rep
        )

        println("β₀ = $b0:")
        println("  α before → after = $(round(as_mean[1], sigdigits=3)) → $(round(as_mean[end], sigdigits=3))\n")

        # choose a color
        col = get(ColorSchemes.RdYlBu_9, (ci-1)/(length(b_range)-1))

        # x coords all = b0
        xs = fill(b0, length(as_mean))

        # plot α vs fixed β₀ with ribbon
        plot!(
          xs, as_mean;
          ribbon    = as_std,
          label     = "β₀ = $b0",
          lw        = 2,
          marker    = :circle,
          color     = col,
        )

        # little arrows showing the step‐by‐step flow
        for i in 1:(length(as_mean)-1)
            dy = as_mean[i+1] - as_mean[i]
            quiver!(
              [b0], [as_mean[i]],
              quiver   = ([0.0], [dy]),
              arrow    = :closed,
              head_width  = 0.02,
              head_length = 0.05,
              lw       = 1,
              color    = col,
              marker   = false,
            )
        end
    end

    savefig("rg_alpha_vs_beta_fixed.pdf")
    display(current())
end

# ========== Run Scan and Plot ==========

lattice_size = 16
a_0 = 3.1
b_0 = sqrt(8π)  # critical value
b_range = [6.5]
#push!(b_range, sqrt(8π))
#sort!(b_range) 
n_samples = 15000
n_steps = 3
Z_0 = 1.0
n_rep = 1
#scan_and_plot_a_vs_b(a_0, b_range, Z_0, lattice_size, n_samples, n_steps, n_rep)
samples = generate_SG_data_hmc_2d(SineGordonModel2D(16, 3.1, sqrt(8π), 1.0),
                                  5000; burn_in=2000, thin=50)

Here we check what the direction of flows for $\alpha$.

In [ ]:
function get_flow_direction(a_0::Float64, b_0::Float64, Z_0::Float64, lattice_size::Int, n_samples::Int, n_steps::Int)
    model = SineGordonModel2D(lattice_size, a_0, b_0, Z_0)
    samples = generate_SG_data_hmc_2d(model, n_samples)
    for step in 1:n_steps
        samples = block_spin_samples(samples)
        a_new, _, _ = estimate_parameters_score_matching_2d(samples)
        return sign(a_new - a_0)  # +1 if α grows, -1 if α shrinks
    end
end

# Phase Diagram
In this code, we diagnose the phase boundary for every value of $\alpha$ and $\beta$, and verify that the boundary lies near $\sqrt{8\pi}$ upto errors

In [ ]:
using Statistics

"""
    estimate_phase_boundary(
      alpha_list::Vector{Float64},
      beta_range::Vector{Float64},
      Z_0::Float64,
      lattice_size::Int,
      n_samples::Int,
      n_steps::Int;
      n_reps::Int = 10
    ) → (α, mean_βc, std_βc, all_βc)

For each repetition rep=1…n_reps, scans β∈sort(beta_range) and finds the first β for which 
`get_flow_direction(a₀,β,…)<0`. Returns:

- α = alpha_list
- mean_βc[i] = mean of βc over reps at α[i]
- std_βc[i]  = std. dev. of βc over reps at α[i]
- all_βc     = n_reps×length(alpha_list) matrix of raw βc’s
"""
function estimate_phase_boundary(
    alpha_list::Vector{Float64},
    beta_range::Vector{Float64},
    Z_0::Float64,
    lattice_size::Int,
    n_samples::Int,
    n_steps::Int;
    n_reps::Int = 10,
)
    nα = length(alpha_list)
    all_βc = Vector{Float64}(undef, nα)

    for (i, a₀) in enumerate(alpha_list)
        println("→ Scanning α = $a₀")
        βc = NaN

        for β₀ in sort(beta_range)
            # do n_reps determinations of the flow sign at (a₀,β₀)
            signs = Int[]
            for rep in 1:n_reps
                push!(signs, get_flow_direction(a₀, β₀, Z_0,
                                                lattice_size,
                                                n_samples,
                                                n_steps))
            end

            # decide by majority: if more than half are negative, we call it 'shrinking'
            if count(<(0), signs) > n_reps ÷ 2
                βc = β₀
                println("    β₀ = $β₀ → majority shrink (", signs, ")")
                break
            else
                println("    β₀ = $β₀ → majority grow  (", signs, ")")
            end
        end

        all_βc[i] = βc
        println("  ⇒ α = $a₀ → β_c ≈ $βc\n")
    end

    mean_βc = mean(all_βc)
    std_βc  = std(all_βc)

    return alpha_list, all_βc, mean_βc, std_βc
end